# Compare Sci Kit Learn and Rapids

In [1]:
import pandas as pd
import numpy as np
import time

from sklearn.ensemble import RandomForestRegressor
from cuml.dask.ensemble import cuRandomForestRegressor
import gc

ModuleNotFoundError: No module named 'cuml'

In [21]:
rnd=17
rf_params = {
    "n_estimators": 500,
    "max_depth": None,
    "min_samples_split": 2,
    "min_samples_leaf":5,
    "warm_start":False,
    "oob_score":True,
    "random_state": rnd,
    "max_features":0.5,
    "n_jobs":10
    }

In [8]:
def dat_create(dat, col, log_col, lt_col, y_col, t_col):

    x_dat = dat[col+t_col+[y_col]].dropna().copy()
    

    if log_col:
       for i in log_col:
            try:
                x_dat[i] = np.log10(x_dat[i])
            except:
                print(f'Could not log column {i}')
    
    if lt_col:
        for i in lt_col:
            try:
                x_dat[f'cos_{i}'] = np.cos(dat[i]*2*np.pi/24.)
                x_dat[f'sin_{i}'] = np.sin(dat[i]*2*np.pi/24.)
            except:
                print(f'Could not add {i} as a cos/sin time column')
    
    x_dat = x_dat[~x_dat.isin([np.nan, np.inf, -np.inf]).any(axis=1)].dropna()
    y_dat = x_dat[y_col].copy()
    x_dat = x_dat.drop(columns=y_col)    
    
    return x_dat, y_dat

In [14]:
col=['1300_02', 'SYM_H index','SatLat']
y_col='400kmDensity' 
t_col=['DateTime'] 
log_col=['1300_02'] 
lt_col=['SatMagLT'] 
rf_params=rf_params 
target_dat='D:\\data\\SatDensities\\satdrag_database_grace_B.hdf5'
sample_sz = 100000

In [15]:
kcol = [col,[y_col],t_col,lt_col]
kflt = [item for sublist in kcol for item in sublist]

In [17]:
df = pd.read_hdf(target_dat)
df = df[kflt].dropna()
df_dat = df[0:sample_sz].copy()

reg_x, reg_y = dat_create(dat=df_dat,col=col,log_col=log_col,lt_col=lt_col,
                              y_col=y_col,t_col=t_col)
reg_y = reg_y*(10**12)

del df
del df_dat
gc.collect

reg_t = reg_x[t_col].copy()
reg_x = reg_x.drop(columns=t_col)


<function gc.collect(generation=2)>

In [26]:
print('Train and fit SciKitLearn model')

start = time.time()
print("Time elapsed working on RandomForest")

rfr = RandomForestRegressor(**rf_params)
rfr.fit(reg_x, reg_y)

end = time.time()
print("Time consumed in working: ",end - start)

Train and fit SciKitLearn model
Time elapsed working on RandomForest
Time consumed in working:  42.08980870246887


In [ ]:
cu_x = reg_x.astype('float32')
cu_y = reg_y.astype('float32')

In [27]:


start = time.time()
print("Time elapsed working on RandomForest")

rf_cu = cuRandomForestRegressor(n_estimators=100, max_depth=10, seed=42)
cu_rfr = rf_cu.fit(cu_x,cu_y)

end = time.time()
print("Time consumed in working: ",end - start)

ModuleNotFoundError: No module named 'cuml'